In [1]:
!pip install -q transformers==4.34.0 accelerate bitsandbytes==0.41.0 peft datasets safetensors
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 2
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 2
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
#Setup and model selection ----------
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch
import os

In [3]:
# Install dependencies (run once per Colab session)
!pip install -q transformers accelerate bitsandbytes -U

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


# Model & tokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,  # 4-bit quantization to fit Free Colab
)

model.to(device)

# Example conversation (replace with your owner-agent messages)
messages = [
    {"role": "user", "content": "hi"},
    {"role": "assistant", "content": "Hello 👋 What do you want me to do? I can add, edit, delete, or modify your product details."},
    {"role": "user", "content": "Can you add a new Laptop with price 1500 and stock 10?"}
]

# Encode chat messages
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

# Generate response
generated_ids = model.generate(
    input_ids=inputs, # Pass the tensor as input_ids
    max_new_tokens=200,   # keep low to avoid OOM
    do_sample=True,
    temperature=0.7
)

# Decode and print assistant reply
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] hi [/INST] Hello 👋 What do you want me to do? I can add, edit, delete, or modify your product details.</s> [INST] Can you add a new Laptop with price 1500 and stock 10? [/INST] Sure! I'll add a new laptop with a price of $1500 and a stock of 10 units. Here's an example of how to add it to your product list:

```python
# Let's assume we have an empty list called 'laptops' to store the product details
laptops = []

# Add a new laptop to the list with price $1500 and stock 10
new_laptop = {
    'name': 'New Laptop',
    'price': 1500,
    'stock': 10
}
laptops.append(new_laptop)

# Print the updated product list
print(laptops)
```

This will add the new laptop to the list with the given name, price, and stock. You can modify the name accordingly.</s>


In [4]:
import json
import random

actions = ["add", "edit", "delete", "list", "offer", "undo", "company_update"]
products = [
    ("Laptop", "LP100"), ("iPhone 15", "IP150"), ("Smartwatch", "SW200"),
    ("Headphones", "HD300"), ("Camera", "CM400"), ("Shoes", "SH500"),
    ("Backpack", "BP600"), ("T-shirt", "TS700"), ("Eco Mug", "EM800"),
    ("Chef Knife", "CK900")
]
tones = ["friendly", "formal", "casual", "professional", "enthusiastic"]

dataset = []

# Always start with greeting dataset
dataset.append({
    "messages": [
        {"role": "user", "content": "hi"},
        {"role": "assistant", "content": "Hello 👋 What do you want me to do? I can add, edit, delete, or modify your product details. I can also manage offers and company details for you."}
    ]
})

for i in range(1000):
    action = random.choice(actions)
    product_name, product_code = random.choice(products)
    tone = random.choice(tones)

    if action == "add":
        user_msg = f"Can you add a new product? name={product_name}, code={product_code}, price={random.randint(100,5000)}, stock={random.randint(5,200)}"
        assistant_msg = f"✅ Product {product_name} ({product_code}) added successfully with the given details. ({tone} tone)"
    elif action == "edit":
        user_msg = f"Update {product_code} price={random.randint(100,5000)}, stock={random.randint(5,200)}"
        assistant_msg = f"✏️ Updated {product_name} ({product_code}) with the new price and stock. ({tone} tone)"
    elif action == "delete":
        user_msg = f"Delete product {product_code}"
        assistant_msg = f"⚠️ Are you sure you want to delete {product_name} ({product_code})? Please confirm."
    elif action == "list":
        user_msg = "Show me electronics products"
        assistant_msg = "Here are some Electronics products: Laptop (LP100), iPhone 15 (IP150), Camera (CM400)."
    elif action == "offer":
        discount = random.randint(5,30)
        user_msg = f"Add offer to {product_code}: {discount}% off"
        assistant_msg = f"🎉 Special offer added! {product_name} ({product_code}) now has {discount}% off. ({tone} tone)"
    elif action == "undo":
        user_msg = "Undo my last action"
        assistant_msg = "↩️ Your last action has been undone successfully. ({tone} tone)"
    elif action == "company_update":
        user_msg = "Update company phone to +94 77 123 4567"
        assistant_msg = "☎️ Company phone updated successfully. ({tone} tone)"

    dataset.append({
        "messages": [
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]
    })

# Save as JSONL for chat fine-tuning
with open("owner_agent_chat_dataset.jsonl", "w", encoding="utf-8") as f:
    for ex in dataset:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")


In [5]:
from huggingface_hub import notebook_login
from google.colab import userdata

# Log in to Hugging Face
notebook_login()

In [6]:
from google.colab import files

# download the file you just saved
files.download("owner_agent_chat_dataset.jsonl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
from datasets import load_dataset

# Load your chat-style dataset
dataset = load_dataset("json", data_files="/content/owner_agent_chat_dataset.jsonl", split="train")

def format_example_chat(ex):
    """
    Converts 'messages' into a single text prompt suitable for supervised fine-tuning.
    Format:
    User: ...
    Assistant: ...
    """
    messages = ex.get("messages", [])
    text = ""
    for msg in messages:
        role = msg["role"].capitalize()  # 'User' or 'Assistant'
        content = msg["content"]
        text += f"{role}: {content}\n"
    return {"text": text}

# Apply formatting
dataset = dataset.map(format_example_chat)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [8]:
def tokenize_chat(example):
    # Use the 'text' column from chat-style formatting
    tokenized = tokenizer(example["text"], truncation=True, max_length=1024)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_chat, batched=False)


Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [9]:
from peft import LoraConfig, get_peft_model
from peft import TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # typical for Mistral/LLaMA
)

model = get_peft_model(model, peft_config)


In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./llama2-owner-agent-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_total_limit=3,
    save_strategy="steps",
    save_steps=50,
)

# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

wandb: Currently logged in as: tharukshidiyunugala96 (tharukshidiyunugala96-university-of-ruhuna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.804500
20,0.732300
30,0.327900
40,0.305800
50,0.266800
60,0.236700
70,0.217600
80,0.223600
90,0.188100
100,0.195600


TrainOutput(global_step=378, training_loss=0.28387016562557726, metrics={'train_runtime': 2195.0638, 'train_samples_per_second': 1.368, 'train_steps_per_second': 0.172, 'total_flos': 5838330615570432.0, 'train_loss': 0.28387016562557726, 'epoch': 3.0, 'num_input_tokens_seen': 0})

In [11]:
model.save_pretrained("./llama2_owner_agent_lora_adapter")
tokenizer.save_pretrained("./llama2_owner_agent_lora_adapter")


('./llama2_owner_agent_lora_adapter/tokenizer_config.json',
 './llama2_owner_agent_lora_adapter/special_tokens_map.json',
 './llama2_owner_agent_lora_adapter/chat_template.jinja',
 './llama2_owner_agent_lora_adapter/tokenizer.model',
 './llama2_owner_agent_lora_adapter/added_tokens.json',
 './llama2_owner_agent_lora_adapter/tokenizer.json')

In [12]:
!pip install -q evaluate

In [14]:
messages = []

def chat_with_owner():
    global messages
    while True:
        user_inp = input("Owner: ")
        if user_inp.lower() in ["quit", "exit"]:
            break
        messages.append({"role": "user", "content": user_inp})

        # tokenize input
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

        # generate response
        outputs = model.generate(inputs, max_new_tokens=100, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # keep only the last assistant reply (after the last "Assistant:")
        if "Assistant:" in reply:
            reply = reply.split("Assistant:")[-1].strip()

        # optional: normalize format (force lowercase first word, remove tone info etc.)
        # example: "✅ Added MacBook (blah...)" → "added macbook price 2000000"
        # here’s a very basic cleaning:
        reply = reply.split("\n")[0]  # only first line
        reply = reply.replace("✅", "").strip()

        print("Assistant:", reply)
        messages.append({"role": "assistant", "content": reply})

chat_with_owner()



Owner: hi
Assistant: 🤝 User added product successfully. (friendly tone)
Owner: Add offer to CK900: 18% off
Assistant: 🎁 Offer added successfully. Chef Knife (CK900) now has 18% off. (professional tone)
Owner: Can you add a new product? name=Backpack, code=BP600, price=4748, stock=50
Assistant: [INST] hi [/INST] 🤝 User added product successfully. (friendly tone) [INST] Add offer to CK900: 18% off [/INST] 🎁 Offer added successfully. Chef Knife (CK900) now has 18% off. (professional tone) [INST] Can you add a new product? name=Backpack, code=BP600, price=4748, stock=50 [/INST]  Product Backpack (BP600) added successfully with the given details. (professional tone)
Owner: exit


In [17]:
# System prompt for product manager assistant
system_prompt = (
    "You are a product manager assistant. "
    "Only respond with confirmations of product actions in the format:\n"
    "Assistant: <action> <product> <details>\n"
    "Do not add emojis, trivia, or extra text."
)

# Initialize messages with system prompt
messages = [{"role": "system", "content": system_prompt}]

def chat_with_owner():
    global messages
    while True:
        user_inp = input("Owner: ")
        if user_inp.lower() in ["quit", "exit"]:
            break

        # Add user message
        messages.append({"role": "user", "content": user_inp})

        # Tokenize messages for model input
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

        # Generate response
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode only new tokens
        reply_tokens = outputs[0][inputs.shape[-1]:]
        reply = tokenizer.decode(reply_tokens, skip_special_tokens=True).strip()

        # Force into fixed template
        if "Assistant:" not in reply:
            reply = "Assistant: " + reply

        # Keep only the first line
        reply = reply.split("\n")[0].strip()

        # Remove extra "Assistant:" if needed
        reply = reply.split("Assistant:")[-1].strip()

        # Clean formatting
        reply = reply.split("Assistant:")[-1].strip()
        reply = reply.split("\n")[0].strip()

        print("Assistant:", reply)

        # Add assistant response to messages
        messages.append({"role": "assistant", "content": reply})

chat_with_owner()

Owner: hello
Assistant: Assisted: Added greeting message. (professional tone)
Owner: add product hexa mobile, cose: rs.80000
Assistant: Assisted: Added product Hexa Mobile with cost: rs.80000. (professional tone)
Owner: edit hexa mobile price to rs.60000
Assistant: Assisted: Updated Hexa Mobile price to rs.60000. (professional tone)
Owner: delete product hexa mobile
Assistant: Assisted: Deleted product Hexa Mobile successfully. (professional tone)
Owner: Update company phone to +94 77 123 4567
Assistant: Assisted: Updated company phone to +94 77 123 4567. (professional tone)
Owner: Undo my last action
Assistant: Assisted: Undone last action. (professional tone)
Owner: exit
